In [ ]:
from langchain.tools import Tool
from openai import OpenAI

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import tool, AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from datetime import datetime, timedelta

load_dotenv()
from dateutil.parser import parse as parse_datetime

import sys

sys.path.append("/app/ica_project2/function-agent/")
from business_sub_agent import create_business_sub_agent
from life_sub_agent import create_life_sub_agent
from search_sub_agent import create_search_sub_agent

# 각 분야별 sub agent
business_agent = create_business_sub_agent(eval_mode=True)
life_agent = create_life_sub_agent(eval_mode=True)
search_agent = create_search_sub_agent(eval_mode=True)


# orchestrator tools
orchestrator_tools = [
    Tool(
        name="business_assitant",
        func=lambda user_input: business_agent.invoke(
            {
                "input": user_input,
            }
        ),
        description=""" 
            해당 도구는 업무를 보조하는 agent 입니다. 
            1. 메일을 검색하거나 메일 초록을 작성하거나, 메일 내용을 요약이 필요할때 활용할 수 있습니다.
            2. 캘린더 일정을 확인하거나 생성, 수정, 삭제하는 등이 필요할때 활용할 수 있습니다.""",
    ),
    Tool(
        name="search_assistant",
        func=lambda user_input: search_agent.invoke(
            {
                "input": user_input,
            }
        ),
        description="""
            해당 도구는 사용자의 요청에 맞는 정보를 검색하는 agent 입니다.
            1. 일반적인 웹 검색이 가능하고.
            2. 특정 도메인에 대한 전문 검색도 가능합니다.
            3. 필요한 정보를 직접 찾아볼 수 있게 링크만 정리해줄 수 도 있습니다.
            """,
    ),
    Tool(
        name="life_assistant",
        func=lambda user_input: life_agent.invoke(
            {
                "input": user_input,
            }
        ),
        description="""
            해당 도구는 생활 편의성을 돕는 어시스턴트입니다.
            1. 사용자가 장소를 검색하고자 할 때 사용할 수 있습니다.
            2. 특정 장소에 대한 날씨 검색 혹은 장소 추천 시 날씨 검색을 할 때 사용할 수 있습니다.
            3. 쇼핑 상품을 검색하거나 장바구니에 저장하고자 할 때 사용할 수 있습니다.
            """,
    ),
]


def create_super_agent(today_str: str):
    now = datetime.now().strftime("%Y-%m-%d")
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"""You are a master AI assistant (super agent). 당신은 사용자의 복잡한 요청을 분석하여, 각 분야의 전문가 어시스턴트에게 작업을 정확히 분배하는 역할을 합니다.
                사용 가능한 전문가 목록은 다음과 같습니다:
                    - business_assitant : 해당 도구는 업무를 보조하는 agent 입니다. 
                    - search_assistant : 해당 도구는 사용자의 요청(장소와 관련된 것을 제외)에 맞는 정보를 검색하는 agent 입니다. 
                    - life_assistant : 해당 도구는 생활 편의성을 돕는 어시스턴트입니다.

                오늘 날짜는 {now} 입니다다
                """,
            ),
            MessagesPlaceholder(variable_name="chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

    agent = create_openai_functions_agent(llm, orchestrator_tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=orchestrator_tools, verbose=True, return_intermediate_steps=True)

    return agent_executor


In [ ]:


from evaluation_data import EVALUATION_SET


load_dotenv()
today_str = datetime.now().strftime("%Y-%m-%d")
super_agent = create_super_agent(today_str = today_str)

for eval_data in EVALUATION_SET:
    
    eval_data = EVALUATION_SET[10]
    query = eval_data['query']
    total_step = len(eval_data['expected_tool_calls'])
    
    today_str = datetime.now().strftime("%Y-%m-%d")
    today_str += f" AM 10:30"

    print(f"\n==================================================")
    print(f"[사용자 쿼리]: {query}")
    print(f"==================================================")

    result = super_agent.invoke({
        "input": query,
        "today": today_str,
        "chat_history": []
    })

    ai_response = result['output']
    print(f"\n[슈퍼 에이전트 최종 답변]: {ai_response}")
    print(result['intermediate_steps'])
    acc_depth_1 = 0
    acc_depth_2 = 0
    for i in range(total_step):
        try:
            each_step = eval_data['expected_tool_calls'][i]
            if each_step['agent_name'] == result['intermediate_steps'][i][0].tool:
                acc_depth_1 += 1
            if each_step['function_name'] == result['intermediate_steps'][i][1]['intermediate_steps'][0][0].tool:
                acc_depth_2 += 1
        except:
            pass
        
    print(f'accuracy : {(acc_depth_1,total_step), (acc_depth_2,total_step)}')
    break



In [ ]:
query


In [ ]:
each_step

In [ ]:
result['intermediate_steps']

In [ ]:
print({'어린왕자라는 책의 줄거리좀 알려줄래?': 'accuracy : (0, 0), (0, 0)', 'LLM과 RAG 차이를 잘 모르겠어 알려줘': 'accuracy : (0, 0), (0, 0)', 'LLM과 RAG의 차이를 검색을 해서 알려줘': 'accuracy : (1, 1), (0, 1)', '오늘 종목 추천해줘': 'accuracy : (0, 1), (0, 1)', '홍길동이랑 주고받은 최근 메일 요약해줘': 'accuracy : (1, 1), (1, 1)', "최신 AI 기술 동향에 대해 TechCrunch에서 검색해서, 그 내용을 바탕으로 우리 팀에게 공유할 메일 초안을 작성해줘. 받는 사람은 'dev_team@mycompany.com' 이야.": 'accuracy : (2, 2), (1, 2)', '오늘 오후 스케쥴에 맞춰 잠깐 한시간정도 쇼핑할만한 장소 추천해줘': 'accuracy : (0, 2), (0, 2)', '이번 주말에 가족과 나들이 갈만한 곳을 추천해줘': 'accuracy : (0, 2), (0, 2)', '오늘 있었던 팀 KPI 회의록 요약해서 박기훈 팀장님에게 보낼 수 있게 메일 내용을 작성해줘': 'accuracy : (0, 2), (0, 2)', '오늘 주말에 민수랑 저녁먹으려고 하는데 강남에 술한잔하기 괜찮은데 알려주고 캘린더에 등록좀 해줘.': 'accuracy : (1, 2), (1, 2)', '주말에 우리 뽀미 데리고 놀러갈건데 강원도에 애견펜션 알려줘. 혹시 그 애견펜션 근처에 강아지 데리고 갈만한 곳들은 없어?, 찾은 곳들로 주말 일정을 등록해줘': 'accuracy : (0, 3), (0, 3)'})

In [ ]:
acc_depth_1

In [ ]:
acc_depth_2

In [ ]:
result['intermediate_steps']